## Authorship Information
__author__ = "Arif Haidari"<br>
__credits__ = ["Bernd Brinkmann", "Luigi Menale", "Alex Tavkhelidze", "Romain Lesieur"]<br>
__status__ = "Development"<br>
__project__ = "Plant Recognition"<br>
__scope__ = "DataScientest's Bootcamp in Data Science"

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
from skimage.feature import hog, local_binary_pattern
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier

In [ ]:
# image classification using Bag of Visual Words (BoVW)
# Special-technique-for-image-classification
# source:
# https://www.kaggle.com/code/shuvoalok/special-technique-for-image-classification#classification-of-image-using-Fourier-Transform-and--Wavelet-Transform

In [ ]:
# requires a lot of time ---> 9 hr +
# I concluded that it is not an optimal method for feature extraction
import cv2
import numpy as np
from sklearn.cluster import KMeans
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your image dataset and labels
# You need to prepare your dataset and labels accordingly

# Step 1: Feature Extraction (e.g., SIFT descriptors)
def extract_bovw(image):
  sift = cv2.SIFT_create()

  gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  keypoint, descriptor = sift.detectAndCompute(gray_image, None)
  return features

# Step 2: Codebook Creation (K-means clustering)
def create_codebook(features, codebook_size):
    kmeans = KMeans(n_clusters=codebook_size)
    kmeans.fit(features)
    codebook = kmeans.cluster_centers_
    return codebook

# Step 3: Generate Histograms of Visual Words
def generate_histograms(features, codebook):
    histograms = []

    for feature in features:
        distances = np.linalg.norm(codebook - feature[:, np.newaxis], axis=2)
        nearest_codewords = np.argmin(distances, axis=1)
        histogram, _ = np.histogram(nearest_codewords, bins=np.arange(len(codebook) + 1))
        histograms.append(histogram)

    return np.array(histograms)

# Step 4: Train a Classifier (e.g., SVM)
def train_classifier(X_train, y_train):
    classifier = SVC(kernel='linear')
    classifier.fit(X_train, y_train)
    return classifier

# Main Code
# extract features from image path
data_dir = '/content/drive/MyDrive/raw_dataset/apple_recognition/'

labels = []
features = []

for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if not os.path.isdir(label_dir):
        continue
    for image_file in os.listdir(label_dir):
        image_path = os.path.join(label_dir, image_file)
        image = cv2.imread(image_path)
        if image is not None:
          # get image features
            hog_features = extract_bovw(image)
            features.append(hog_features)
            labels.append(label)

# Convert to numpy arrays
# X = np.array(features)
# target = np.array(labels)

le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

X = np.vstack(features)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels_encoded, test_size=0.2, random_state=42)

# Create the codebook
codebook_size = 100  # Number of visual words
codebook = create_codebook(X_train, codebook_size)

# Generate histograms of visual words for training and testing data
X_train_hist = generate_histograms(X_train, codebook)
X_test_hist = generate_histograms(X_test, codebook)

# Train a classifier
classifier = train_classifier(X_train_hist, y_train)

# Predict labels for test data
y_pred = classifier.predict(X_test_hist)

# Evaluate the classifier
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

In [ ]:
# Classification of images using Fourier Transform and Wavelet Transform involves
# extracting features from images in the frequency domain and then using these features for 
# classification.

# Since this method create a lot dimension, we use PCA to reduce the dimensionality.

In [ ]:

import pywt
from sklearn.decomposition import PCA

# Step 1: Feature Extraction - Fourier Transform
def extract_fourier_features(image_path, size=(128, 128)):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, size)
    # Apply 2D Fourier Transform
    f_transform = np.fft.fft2(image)
    f_transform_shifted = np.fft.fftshift(f_transform)
    magnitude_spectrum = np.log(np.abs(f_transform_shifted) + 1)  # Log magnitude
    return magnitude_spectrum.flatten()

# Step 2: Feature Extraction - Wavelet Transform
def extract_wavelet_features(image_path, wavelet='haar', level=1, size=(128, 128)):
    image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, size)
    # Apply 2D Wavelet Transform
    coeffs = pywt.wavedec2(image, wavelet, level=level)
    features = np.hstack([c.flatten() for c in coeffs[0:1] + sum([list(c) for c in coeffs[1:]], [])])
    return features

# Prepare feature and label lists
fourier_features = []
wavelet_features = []
labels = []

data_dir = '/content/drive/MyDrive/raw_dataset/apple_recognition/'

for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if not os.path.isdir(label_dir):
        continue
    for image_file in os.listdir(label_dir):
        image_path = os.path.join(label_dir, image_file)
        if image_path is not None:
            # Extract features from image
            fourier = extract_fourier_features(image_path)
            wavelet = extract_wavelet_features(image_path)
            fourier_features.append(fourier)
            wavelet_features.append(wavelet)
            labels.append(label)

# Convert feature lists to numpy arrays
fourier_features = np.array(fourier_features)
wavelet_features = np.array(wavelet_features)

# Apply PCA to standardize feature dimensions
pca_fourier = PCA(n_components=100)  # Adjust the number of components as needed
pca_wavelet = PCA(n_components=100)  # Adjust the number of components as needed
fourier_features_pca = pca_fourier.fit_transform(fourier_features)
wavelet_features_pca = pca_wavelet.fit_transform(wavelet_features)

# Combine Fourier and Wavelet features
X = np.hstack((fourier_features_pca, wavelet_features_pca))

# Encode labels
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, labels_encoded, test_size=0.2, random_state=42)

# Step 4: Train a Classifier (e.g., SVM)
def train_classifier(X_train, y_train):
    classifier = SVC(kernel='linear')
    classifier.fit(X_train, y_train)
    return classifier

# Train the classifier
classifier = train_classifier(X_train, y_train)

# Step 5: Predict and Evaluate
# Predict labels for test data
y_pred = classifier.predict(X_test)

# Evaluate the classifier
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))


Accuracy: 0.7146739130434783
Classification Report:
                          precision    recall  f1-score   support

      Apple___Apple_scab       0.54      0.70      0.61        99
       Apple___Black_rot       0.68      0.65      0.67        75
Apple___Cedar_apple_rust       0.84      0.72      0.77        95
         Apple___healthy       0.89      0.78      0.83        99

                accuracy                           0.71       368
               macro avg       0.74      0.71      0.72       368
            weighted avg       0.74      0.71      0.72       368



In [ ]:
# SIFT (Scale-Invariant Feature Transform): Detects and describes local features in images that are invariant to scale and rotation.
# SURF (Speeded-Up Robust Features): Similar to SIFT but faster.
# ORB (Oriented FAST and Rotated BRIEF): A fast and efficient alternative to SIFT and SURF.

# Since ORB is the optimal method among three of them then we only continue with ORB

In [ ]:
def extract_features_orb(image):
    orb = cv2.ORB_create()
    keypoints, descriptors = orb.detectAndCompute(image, None)
    if descriptors is None:
        descriptors = np.array([])
    # Pad descriptors to a fixed length (e.g., 500)
    if descriptors.shape[0] < 500:
        descriptors = np.pad(descriptors, ((0, 500 - descriptors.shape[0]), (0, 0)), mode='constant')
    return descriptors.flatten()

data_dir = '/content/drive/MyDrive/raw_dataset/apple_recognition/'

labels = []
features = []

for label in os.listdir(data_dir):
    label_dir = os.path.join(data_dir, label)
    if not os.path.isdir(label_dir):
        continue
    for image_file in os.listdir(label_dir):
        image_path = os.path.join(label_dir, image_file)
        image = cv2.imread(image_path)
        if image is not None:
            gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            # get image features
            orb_features = extract_features_orb(gray_image)
            features.append(orb_features)
            labels.append(label)

# Convert to numpy arrays
X = np.array(features)
target = np.array(labels)

le = LabelEncoder()
labels_encoded = le.fit_transform(target)

X_train, X_test, y_train, y_test = train_test_split(X, labels_encoded, test_size=0.2, random_state=42)

# XGBoost model
xgb_model = XGBClassifier(n_estimators=100,
                               max_depth=3,
                               learning_rate=0.1,
                               n_jobs=-1,
                               subsample=0.8,
                               colsample_bytree=0.8,
                               objective='multi:softmax',
                               num_class=len(np.unique(labels_encoded)))

# Train the model
xgb_model.fit(X_train, y_train, eval_metric='mlogloss', eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)

# Predictions
y_pred = xgb_model.predict(X_test)

# Evaluation
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:")
print(classification_report(y_test, y_pred, target_names=le.classes_))

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


Accuracy: 0.5733695652173914
Classification Report:
                          precision    recall  f1-score   support

      Apple___Apple_scab       0.51      0.43      0.47        99
       Apple___Black_rot       0.53      0.64      0.58        75
Apple___Cedar_apple_rust       0.64      0.64      0.64        95
         Apple___healthy       0.61      0.60      0.60        99

                accuracy                           0.57       368
               macro avg       0.57      0.58      0.57       368
            weighted avg       0.57      0.57      0.57       368

